In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Let's import our dataset
insurance = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv")
insurance

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [5]:
# In order to prepare our dataset we need to import some libraries from sklearn
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# Let's Create a column transformer
col_transform = make_column_transformer(
    (MinMaxScaler(),["age","bmi","children"]),
    (OneHotEncoder(handle_unknown="ignore"),["sex","smoker","region"])
)

# Create X and y
X = insurance.drop("charges",axis= 1)
y = insurance["charges"]

# Build our training and testing data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Fit the column transformer to our training data
col_transform.fit(X_train)

# transform training and test data with normalization (MinMaxScaler) and OneHotEncoder
X_train_normal = col_transform.transform(X_train)
X_test_normal = col_transform.transform(X_test)

In [6]:
# Let's visualize before and after data
X_train.loc[0]

age                19
sex            female
bmi              27.9
children            0
smoker            yes
region      southwest
Name: 0, dtype: object

In [7]:
X_train_normal[0]

array([0.60869565, 0.10734463, 0.4       , 1.        , 0.        ,
       1.        , 0.        , 0.        , 1.        , 0.        ,
       0.        ])

In [8]:
# Let's check for more samples
X_train_normal

array([[0.60869565, 0.10734463, 0.4       , ..., 1.        , 0.        ,
        0.        ],
       [0.63043478, 0.22491256, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.73913043, 0.23944041, 0.        , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.86956522, 0.24791499, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.41304348, 0.85122411, 0.4       , ..., 0.        , 0.        ,
        1.        ],
       [0.80434783, 0.37503363, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [9]:
X_train.shape, X_train_normal.shape # new columns are created as the data is one hot encoded

((1070, 6), (1070, 11))

## Let's build a neural network model

In [12]:
# Set random seed
tf.random.set_seed(42)

# 1. Create a model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(1)
])

# 2. Compile the model
model.compile(loss=tf.keras.losses.mae,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['mae'])

# 3. Fit the model
model.fit(X_train_normal, y_train, epochs=100)

Epoch 1/100
34/34 [==============================] - 0s 749us/step - loss: 13344.8936 - mae: 13344.8936
Epoch 2/100
34/34 [==============================] - 0s 636us/step - loss: 13340.6152 - mae: 13340.6152
Epoch 3/100
34/34 [==============================] - 0s 789us/step - loss: 13327.9893 - mae: 13327.9893
Epoch 4/100
34/34 [==============================] - 0s 636us/step - loss: 13297.4453 - mae: 13297.4453
Epoch 5/100
34/34 [==============================] - 0s 697us/step - loss: 13238.0293 - mae: 13238.0293
Epoch 6/100
34/34 [==============================] - 0s 645us/step - loss: 13138.6172 - mae: 13138.6172
Epoch 7/100
34/34 [==============================] - 0s 638us/step - loss: 12988.4170 - mae: 12988.4170
Epoch 8/100
34/34 [==============================] - 0s 606us/step - loss: 12776.7793 - mae: 12776.7793
Epoch 9/100
34/34 [==============================] - 0s 606us/step - loss: 12493.0518 - mae: 12493.0518
Epoch 10/100
34/34 [==============================] - 0s 693us/s

In [13]:
# Let's Evaluate our model
model.evaluate(X_test_normal,y_test)

9/9 [==============================] - 0s 881us/step - loss: 3444.9248 - mae: 3444.9248


[3444.9248046875, 3444.9248046875]